<a href="https://colab.research.google.com/github/ronen-halevy/ai-notebooks/blob/main/eager_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:


import tensorflow as tf


def main():

    (train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

    train_images, test_images = train_images / 255.0, test_images / 255.0

    batch_size = 32
    train_dataset = tf.data.Dataset.from_tensor_slices(
        (tf.cast(train_images, tf.float32),
         tf.cast(train_labels, tf.int64)))
    train_dataset = train_dataset.shuffle(50000).batch(batch_size)

    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(10, activation='softmax'),
    ])
    model.summary()


    optimizer = tf.keras.optimizers.Adam()
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
    loss_metric = tf.keras.metrics.Mean(name='train_loss')
    accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

    train_loss_history = []
    accuracy_history = []

    # @tf.function
    def train_step(images, labels):
        with tf.GradientTape() as tape:
            predictions = model(images, training=True)
            pred_loss = loss_fn(labels, predictions)

        grads = tape.gradient(pred_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        loss_metric.update_state(pred_loss)
        accuracy_metric.update_state(labels, predictions)

    for epoch in range(2):
        loss_metric.reset_states()
        accuracy_metric.reset_states()

        for images, labels in train_dataset:
            train_step(images, labels)

        train_loss_history.append(loss_metric.result())
        accuracy_history.append(accuracy_metric.result())
        print(f'Epoch {epoch}, Loss {loss_metric.result()}, Accuracy {accuracy_metric.result()}')

    print("accuracy: ", accuracy_history)
    print("train loss: ", train_loss_history)


if __name__ == '__main__':
    main()

eager mode True
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 15, 15, 32)       0         
 2D)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 7200)              0         
                                                                 
 dense_11 (Dense)            (None, 10)                72010     
                                                                 
Total params: 72,906
Trainable params: 72,906
Non-trainable params: 0
_________________________________________________________________
Epoch 0, Loss 1.4632290601730347, Accuracy 0.48848000168800354
Epoch 1, Loss 1.1753791570663452, Acc